In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt

from regression_tools.plotting_tools import (
    plot_univariate_smooth,
    bootstrap_train,
    display_coef,
    plot_bootstrap_coefs,
    plot_partial_depenence,
    plot_partial_dependences,
    predicteds_vs_actuals)

import pandas as pd
%matplotlib inline

In [2]:
def create_machine_type_feature(df):
    temp = pd.DataFrame(df['fiProductClassDesc'].str.split('-',1).tolist(),columns = ['MachineType','MachineSubType'])
    df['MachineType'] = temp['MachineType']
    df = df.drop(['fiProductClassDesc'], axis=1)
    df = pd.concat([df, pd.get_dummies(df['MachineType'])], axis=1)
    df = df.drop(['MachineType'], axis=1)
    return df


def create_age_in_years_feature(df):
    '''
    Creates 'AgeInYears' column.
    Drops 'saledate' and 'YearMade' columns.
    Returns resulting data frame.
    '''
    # Create column: 'AgeInYears' = 'saledate' - 'YearMade'
    df['AgeInYears'] = pd.DatetimeIndex(df['saledate']).year - df['YearMade']
    # Drop columns 'saledate' and 'YearMade'
    df = df.drop(['saledate', 'YearMade'], axis=1)
    # return df
    return df


def standardize_X(X_train, X_test):
    scaler = StandardScaler()
    # Use X_train to compute mean and standard deviation
    scaler.fit(X_train)
    # Standardize X_train
    X_train_standardized = scaler.transform(X_train)
    # Standardize X_test
    X_test_standardized = scaler.transform(X_test)
    return X_train_standardized, X_test_standardized


def preprocess(df_train, df_test):
    
    # Drop rows with 'YearMade' of 1000
    df_train.drop(df_train[df_train['YearMade']==1000].index,inplace=True)
#     df_test.drop(df_test[df_test['YearMade']==1000].index,inplace=True)
    
    df_train = create_machine_type_feature(df_train)
    df_test = create_machine_type_feature(df_test)
    
    # Create 'AgeInYears' column. Drop 'saledate' and 'YearMade' columns.
    df_train = create_age_in_years_feature(df_train)
    df_test = create_age_in_years_feature(df_test)
    
    X_train = df_train.as_matrix()
    X_test = df_test.as_matrix()
    
    X_train, X_test = standardize_X(X_train, X_test)
    
    return X_train, X_test
    
    
# Read in columns we want
df_train = pd.read_csv('data/Train.csv',usecols=["saledate", "YearMade",'fiProductClassDesc'])
df_test = pd.read_csv('data/test.csv',usecols=["saledate", "YearMade",'fiProductClassDesc'])

y_train = pd.read_csv('data/Train.csv',usecols=["SalePrice","YearMade"])
# Drop rows with 'YearMade' of 1000
y_train.drop(y_train[y_train['YearMade']==1000].index,inplace=True)
y_train.drop(['YearMade'], axis=1,inplace=True)
y_train = y_train.as_matrix()

y_test = pd.read_csv('data/end_of_day/test_actual.csv',usecols=["SalePrice"]).as_matrix()

X_train, X_test = preprocess(df_train, df_test)

# print('TRAIN X: ', X_train)
# print('TEST X: ', X_test)
# print('TRAIN y: ', y_train)
# print('TEST y: ', y_test)

FileNotFoundError: File b'data/Train.csv' does not exist

In [ ]:
np.shape(X_train)

In [ ]:
np.shape(X_test)

In [ ]:
np.shape(y_train)

In [ ]:
np.shape(y_test)

In [ ]:
# quick fit
model = Ridge(alpha=330)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_hat,y_test))


In [ ]:
RMSE

In [ ]:
def plot_univariate(df,cols,dep_val):
    univariate_plot_names = cols
    if len(univariate_plot_names)%2 ==0:
        upn_len = int(len(univariate_plot_names)/2)
    else:
        upn_len = int(len(univariate_plot_names)/2+1)
    fig, axs = plt.subplots(upn_len, 2, figsize=(14, 8))

    for name, ax in zip(univariate_plot_names, axs.flatten()):
        plot_univariate_smooth(ax,
                               df[name].values.reshape(-1, 1).astype(np.float64),
                               df[dep_val],
                               bootstrap=100)
        ax.set_title(name)
    fig.tight_layout()

In [ ]:
full_df = pd.concat([pd.DataFrame(df_train), pd.DataFrame(y_train)], axis=1)
full_df.drop(['MachineType'], axis=1, inplace=True)
cols = [x for x in full_df.columns]
dep_val = cols[-1]
plot_univariate(full_df,cols[:-1],dep_val)

In [ ]:
full_df.info()

In [ ]:
def rmsle(actual, predictions):
    log_diff = np.log(predictions+1) - np.log(actual+1)
    return np.sqrt(np.mean(log_diff**2))

In [ ]:
def cross_val(X, y, model, n_folds, errortype='RMSE', random_seed=154):
    """Estimate the in- and out-of-sample error of a model using cross
    validation.
    
    Requirements
    ----------
    class XyScaler
    function rmsle
 
    Parameters
    ----------
 
    X: np.array
      Matrix of predictors, standardized
 
    y: np.array
      Target array, standardized
 
    model: sklearn model object.
      The estimator to fit.  Must have fit and predict methods.
 
    n_folds: int
      The number of folds in the cross validation.
    
    errortype: string
      either 'RMSE' or 'RMSLE'
 
    random_seed: int
      A seed for the random number generator, for repeatability.
 
    Returns
    -------
 
    errors, cfs_best: tuple of arrays
      errors = (train errors, test errors) for each fold of cross-validation
      cfs-best = coefficients selected from minimum test error
    """
    kf = KFold(n_folds)
    errorlist = []
    cfs = []
    for k, (train_index,test_index) in enumerate(kf.split(X)):
        # define variables
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        # fit model
        model.fit(X_train,y_train)
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test) 
        # evaluate model
        if errortype == 'RMSE':
            rmse_train = np.sqrt(mean_squared_error(y_hat_train,y_train))
            rmse_test = np.sqrt(mean_squared_error(y_hat_test,y_test))
            errorlist.append((rmse_train, rmse_test)) # tuple output
        elif errortype == 'RMSLE':
            rmsle_train = rmsle(y_train, y_hat_train)
            rmsle_test = rmsle(y_test, y_hat_test)
            errorlist.append((rmsle_train, rmsle_test)) # tuple output
        # store coefficients
        cfs.append (model.coef_)
    # select best coefficients 
    errors = np.asarray(errorlist)
    idx_min_test_error = errors[:,1].argmin()   
    cfs_best = cfs[idx_min_test_error]
    
    return(errors, cfs_best)

In [ ]:
def train_at_various_alphas(X, y, model, alphas, n_folds=10, errortype='RMSE', **kwargs):
    """Train a regularized regression model using cross validation at various
    values of alpha.
    
    requirements
    ----------
    class XyScaler
    function rmsle
    function cross_val

    Parameters
    ----------
 
    X: np.array
      Matrix of predictors, standardized
 
    y: np.array
      Target array, standardized
 
    model: name of sklearn model class
      A class in sklearn that can be used to create a regularized regression
      object.  Options are `Ridge` and `Lasso`.
 
    alphas: numpy array
      An array of regularization parameters.
 
    n_folds: int
      Number of cross validation folds.
    
    errortype: string
      either 'RMSE' or 'RMSLE'
 
    Returns
    -------
 
    cv_errors_train, cv_errors_test: tuple of DataFrame
      DataFrames containing the training and testing errors for each value of
      alpha and each cross validation fold.  Each row represents a CV fold, and
      each column a value of alpha.
      
      Dataframe containing coefficients for each parameter for each alpha
    """
    cv_errors_train = pd.DataFrame(np.empty(shape=(n_folds, len(alphas))),
                                     columns=alphas)
    cv_errors_test = pd.DataFrame(np.empty(shape=(n_folds, len(alphas))),
                                        columns=alphas)
    coefs_df = pd.DataFrame(np.empty(shape=(n_folds, len(alphas))),
                                        columns=alphas)
    for idx, alpha in enumerate(alphas):
        errors, coefs = cross_val(X, y, model(alpha), n_folds, errortype='RMSE', random_seed=154)
        cv_errors_train.iloc[:,idx] = errors[:,0]
        cv_errors_test.iloc[:,idx] = errors[:,1]
        #coefs_df.iloc[:,idx] = coefs
    return(cv_errors_train, cv_errors_test)#, coefs_df

In [ ]:
X_sub = X_train[:1000]
y_sub = y_train[:1000]

In [ ]:
len(X_sub)

In [ ]:
model = Lasso(alpha=330)
n_folds = 2
errortype = 'RMSE'
errors, coefs = cross_val(X_sub, y_sub, model, n_folds, errortype, random_seed=154)

In [ ]:
errors

In [ ]:
X = X_sub
y = y_sub
model = Lasso # must be 'Ridge' or 'Lasso'
alphas = np.logspace(-1,5,30)
n_folds=5
errortype='RMSE'
cv_errors_train, cv_errors_test = train_at_various_alphas(X, y, model, alphas, n_folds, errortype)

In [ ]:
# outputs from alpha testing
train_means = cv_errors_train.mean(axis=0)
test_means = cv_errors_test.mean(axis=0)
# calc min alpha
min_idx = test_means.values.argmin()
min_alpha = alphas[min_idx]
print('min error = {}'.format(np.min(test_means)))
print('alpha at min error = {}'.format(min_alpha))

fig, axs = plt.subplots(1,1,figsize=(5,5));
h1 = axs.plot(alphas,train_means,'b-')
h2 = axs.plot(alphas,test_means,'k-')
axs.set_xscale('log')
axs.invert_xaxis()
axs.set_xlabel('alpha')
axs.set_ylabel('error')
plt.axvline(min_alpha, linestyle='dashed')
plt.legend(('train','test'));
#axs.set_yscale('log');

In [ ]:
# plot: coefficients vs lambda

# fig, axs = plt.subplots(1,1,figsize=(6,6));
# axs.set_xlabel('alpha')
# axs.set_ylabel('coefficient')

# for idx, a in enumerate(alphas):
#     h1 = axs.scatter(alphas[idx],coefs_df[a][0])
#     h2 = axs.scatter(alphas[idx],coefs_df[a][1])
#     h3 = axs.scatter(alphas[idx],coefs_df[a][2])
#     h4 = axs.scatter(alphas[idx],coefs_df[a][3])
#     h5 = axs.scatter(alphas[idx],coefs_df[a][4])
# plt.legend(('c1','c2','c3','c4','c5'));
# axs.set_xscale('log');

In [ ]:
def boot(X, y, size=10000):
    index_list = np.arange(len(X))
    coefs = []
    for i in range(size):
        idxs = np.array([np.random.choice(index_list) for v in X])
        X_data = X[idxs]
        y_data = y[idxs]
        model = Ridge(alpha=.5)
        model.fit(X_data,y_data)
        coefs.append(model.coef_)
    return coefs

In [ ]:
def make_arrays(lol_cofs):
    coef1 = []
    idx = 0
    for row in lol_cofs:
        coef1.append(row[idx])
    return coef1
    plt.hist(make_arrays, normed=True, bins=30)

In [ ]:
cof_boot = boot(X_test, y_test, size=50)
array = make_arrays(cof_boot);
plt.hist(array, normed=True, bins=30);


In [ ]:
# ideas
# can use lasso to pick params:
# e.g. lasso = Lasso(alpha=a), lasso.fit(X,y), newX = [x for x in lasso.coefs_ where x > 0]